In [1]:
import pandas as pd
import numpy as np
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.ensemble import RandomForestRegressor
from scipy.stats import gmean, ttest_ind
from statsmodels.stats.multitest import multipletests
import warnings
warnings.filterwarnings('ignore')

# should have done before using data for Deseq2, not only na values but also check for zero within the cells

In [27]:
data = pd.read_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_Eto_vs_FTY.xlsx")
data1 = pd.read_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_Eto_vs_Eto.xlsx")
data2 = pd.read_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_vs_Eto.xlsx")
data3 = pd.read_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_Eto_vs_CTRL.xlsx")
data4 = pd.read_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_vs_CTRL.xlsx")
data5 = pd.read_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_Eto_vs_CTRL.xlsx")

In [28]:
data5.info()
data5.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14196 entries, 0 to 14195
Data columns (total 9 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   Gene    14196 non-null  object 
 1   CTRL-1  14196 non-null  float64
 2   CTRL-2  14196 non-null  float64
 3   CTRL-3  14196 non-null  float64
 4   Eto-1   14196 non-null  float64
 5   Eto-2   14196 non-null  float64
 6   Eto-3   14196 non-null  float64
 7   p_adj   13922 non-null  float64
 8   log2FC  14196 non-null  float64
dtypes: float64(8), object(1)
memory usage: 998.3+ KB


Gene        0
CTRL-1      0
CTRL-2      0
CTRL-3      0
Eto-1       0
Eto-2       0
Eto-3       0
p_adj     274
log2FC      0
dtype: int64

In [29]:
(data5 == 0).sum().sum()

28

In [30]:
data5[(data5 == 0).any(axis=1)]

,Gene,CTRL-1,CTRL-2,CTRL-3,Eto-1,Eto-2,Eto-3,p_adj,log2FC
269,SPC25,97.389313,159.024495,161.546016,5.513711,0.000000,0.000000,5.437164e-14,-6.180205
300,ERCC6L,106.615669,135.075023,100.565069,5.513711,1.113384,0.000000,1.982515e-13,-5.630237
402,DEPDC1B,64.584492,82.386184,84.517452,2.205484,2.226767,0.000000,3.387765e-11,-5.642962
514,E2F8,39.980876,63.226606,75.958723,0.000000,0.000000,0.000000,9.910861e-10,-8.121398
558,FAM111B,112.766573,128.369171,164.755539,0.000000,1.113384,0.000000,2.178889e-09,-8.339841
975,TMPO-AS1,33.829972,26.823409,37.444441,1.102742,0.000000,2.578115,1.369899e-06,-4.806293
1107,MEMO1P1,22.553315,41.193092,63.120629,2.205484,0.000000,12.890575,3.488237e-06,-3.142939
1204,AL031777.3,1.025151,3.831916,0.000000,9.924679,24.494441,28.359265,7.368760e-06,3.618602
1612,PRC1,12.301808,22.033514,26.746029,5.513711,1.113384,0.000000,7.394733e-05,-3.138749
1707,FBXO43,20.503013,17.243620,20.326982,0.000000,0.000000,2.578115,1.295266e-04,-4.634932


In [31]:
print((data5 == 0).sum().sum(), 
      (data4 == 0).sum().sum(),
     (data3 == 0).sum().sum(),
     (data2 == 0). sum().sum(),
      (data1 == 0).sum().sum(),
      (data == 0).sum().sum())


28 232 6 250 24 228


In [32]:
data3.head()

,Gene,CTRL-1,CTRL-2,CTRL-3,FTY-Eto-1,FTY-Eto-2,FTY-Eto-3,p_adj,log2FC
0,FN1,392645.005013,310656.266920,300990.182521,420019.937412,438405.899710,331730.863338,0.118923,0.244972
1,MT-CO1,177918.997838,161009.427241,154033.590905,188614.990231,200170.604639,163848.571983,0.232942,0.164845
2,COL1A1,242785.405920,286263.250559,266464.268522,237918.762266,230388.063359,192954.186821,0.002628,-0.266673
3,COL1A2,175690.320301,193656.389679,180014.683539,181571.140818,179660.885660,168745.061810,0.577944,-0.051841
4,IGFBP3,139307.723338,121186.244971,118565.146843,150509.648061,145129.656030,106839.579381,0.754401,0.086484


In [33]:
control_cols = ["CTRL-1", "CTRL-2", "CTRL-3"]
fty_cols = ["FTY-1", "FTY-2", "FTY-3"]
eto_cols = ["Eto-1","Eto-2","Eto-3"]
fyt_eto_cols = ["FTY-Eto-1", "FTY-Eto-2", "FTY-Eto-3"]

In [34]:
def filter_rows(row):
    control_zero_count = (row.iloc[1:4] == 0).sum()
    fty_zero_count = (row.iloc[4:7] == 0).sum()
    return not ((control_zero_count > 1) or (fty_zero_count > 1))

filtered_data = data[data.apply(filter_rows, axis=1)]
filtered_data1 = data1[data1.apply(filter_rows, axis=1)]
filtered_data2 = data2[data2.apply(filter_rows, axis=1)]
filtered_data3 = data3[data3.apply(filter_rows, axis=1)]
filtered_data4 = data4[data4.apply(filter_rows, axis=1)]
filtered_data5 = data5[data5.apply(filter_rows, axis=1)]

In [37]:
print((filtered_data5 == 0).sum().sum(), 
      (filtered_data4 == 0).sum().sum(),
     (filtered_data3 == 0).sum().sum(),
     (filtered_data2 == 0). sum().sum(),
      (filtered_data1 == 0).sum().sum(),
      (filtered_data == 0).sum().sum())

19 230 6 239 15 226


In [38]:
def impute_group(group):
    imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)
    imputed_group = imputer.fit_transform(group)
    return pd.DataFrame(imputed_group, columns=group.columns, index=group.index)

control_imputed = impute_group(filtered_data.iloc[:, 1:4])
filtered_data.iloc[:, 1:4] = control_imputed

treatment_imputed = impute_group(filtered_data.iloc[:, 4:7])
filtered_data.iloc[:, 4:7] = treatment_imputed


(filtered_data == 0).sum().sum()











226

In [39]:
def impute_group(group):
    group.replace(0, np.nan, inplace=True)
    imputer = IterativeImputer(estimator=RandomForestRegressor(), max_iter=10, random_state=0)
    imputed_group = imputer.fit_transform(group)
    return pd.DataFrame(imputed_group, columns=group.columns, index=group.index)

control_imputed = impute_group(filtered_data.iloc[:, 1:4])
filtered_data.iloc[:, 1:4] = control_imputed

treatment_imputed = impute_group(filtered_data.iloc[:, 4:7])
filtered_data.iloc[:, 4:7] = treatment_imputed

(filtered_data == 0).sum().sum()



0

In [52]:
cleaned_filtered_data5.shape

(13904, 9)

In [43]:
filtered_data5.replace(0, np.nan, inplace=True)
cleaned_data5 = filtered_data5.dropna()
cleaned_data5.reset_index(drop=True, inplace=True)
cleaned_data5.shape

(13904, 9)

In [53]:
def remove_rows_with_zero_or_na(*dfs):
    cleaned_dfs = []
    for df in dfs:
        df.replace(0, np.nan, inplace=True)  
        cleaned_df = df.dropna()  
        cleaned_df.reset_index(drop=True, inplace=True) 
        cleaned_dfs.append(cleaned_df)
    return cleaned_dfs

cleaned_filtered_data, cleaned_filtered_data1, cleaned_filtered_data2, cleaned_filtered_data3, cleaned_filtered_data4, cleaned_filtered_data5 = remove_rows_with_zero_or_na(
    filtered_data, filtered_data1, filtered_data2, filtered_data3, filtered_data4, filtered_data5)


In [54]:
cleaned_filtered_data.to_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_Eto_vs_FTY.xlsx", index=False)
cleaned_filtered_data1.to_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_Eto_vs_Eto.xlsx", index=False)
cleaned_filtered_data2.to_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_vs_Eto.xlsx", index=False)
cleaned_filtered_data3.to_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_Eto_vs_CTRL.xlsx", index=False)
cleaned_filtered_data4.to_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_FTY_vs_CTRL.xlsx", index=False)
cleaned_filtered_data5.to_excel("/Users/mortezaabyadeh/Desktop/clean_transcriptome_Eto_vs_CTRL.xlsx", index=False)